# flowField class for iterative solution

In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from flowField import *
from flowFieldWavy import *
import h5py
import numpy as np
import pdb
import cProfile
import time
from pseudo import *
from scipy.linalg import norm
from scipy.sparse.linalg import gmres, LinearOperator, lgmres

import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
%run test_flowFieldWavy.py

..........

Note: when testing for derivatives, if looking at the last modes in x or z,remember that the general formulae I use do not apply, because one/some of their neighbouring modes are missing



----------------------------------------------------------------------
Ran 10 tests in 0.140s

OK


In [4]:
tempDict = getDefaultDict()
L = 23; M = 23; N=35
tempDict.update({'K':0, 'L':L,'M':M,'N':N, 'eps':1.0e-3, 'alpha':20.,'beta':50.,'nd':4,'isPois':1, 'Re':100.})
x00 = flowFieldWavy(flowDict=tempDict).view4d()
x00[0,L,M,0] = 1.-(x00.y)**2
vF = x00.slice(nd=[0,1,2])
pF = x00.getScalar(nd=2)




In [5]:
start = time.time()
print(vF.residuals(pField=pF)[0].norm())
print(time.time()-start)

0.0704367978939
4.496234893798828


## Using numpy arrays for gmres instead of flowFieldWavy
If I do that, I can use scipy's gmres and other routines. The plan is this- supply weighted np.ndarray copies of flowFieldWavy instances, and when the Jacobian matvec needs to be computed, make a flowField instance (actually, just assign to an existing instance) out of the array. 

In [33]:
tempDict = getDefaultDict()
L = 3; M = 5; N=31
eps=1.0e-02; gz = 0.8; b= gz/eps; gx= 0.05; a = gx/eps;
tempDict.update({'K':0, 'L':L,'M':M,'N':N, 'eps':eps, 'alpha':a,'beta':b,'nd':4,'isPois':1, 'Re':100.})
x00 = flowFieldWavy(flowDict=tempDict).view4d()
x00[0,L,M,0] = 1.-(x00.y)**2
pF = x00.getScalar(nd=2)
print(x00.shape)
x0 = x00.copy()

(1, 7, 11, 4, 31)


In [4]:
def vecnorm2(vec,N):
    m = vec.size//N
    w = clencurt(N).reshape((1,N))
    W = np.tile(w,(1,m))
    tempVec = vec.reshape(vec.size,1)
    tempVecConj = tempVec.conj().T
    
    vecNorm = np.sqrt(np.abs( np.dot( tempVecConj*W, tempVec ) ))
    return vecNorm[0,0]

#flowDict = tempDict
finDifEps =1.0e-7
def residual(xF):
    #return (xF.slice(nd=[0,1,2]).residuals(pField=xF.getScalar(nd=3))[0]).appendField(xF.slice(nd=[0,1,2]).div())
    #xTemp = xF.slice(L=xF.nx//2+2,M=xF.nz//2+2)
    xTemp = xF.slice(L=xF.nx//2+2)
    resid = (xTemp.slice(nd=[0,1,2]).residuals(pField=xTemp.getScalar(nd=3))[0]).appendField(xTemp.slice(nd=[0,1,2]).div())
    #return resid.slice(L=xF.nx//2,M=xF.nz//2)
    return resid.slice(L=xF.nx//2)

def jacobian(x0,r0,arr):
    xFF = weighted2ff(flowDict=x0.flowDict, arr=arr)
    #vf = finDifEps*xFF.slice(nd=[0,1,2]) + x0.slice(nd=[0,1,2])
    #pf = finDifEps*xFF.getScalar(nd=3) + x0.slice(nd=3)
    #residual = vf.residuals(pField=pf)[0]
    #residual= residual.appendField(vf.div())
    
    matVecProd = (residual(x0+finDifEps*xFF)-r0)/finDifEps
    return matVecProd.weighted()

searchArr = np.append(np.arange(0.,2.1,0.1),np.array([2.5,3.0,4.0,5.0]))
searchArr = np.append(-searchArr[::-1],searchArr[1:])
def lineSearch(x0,arr):
    delFF = (weighted2ff(flowDict=x0.flowDict,arr=arr).view4d())
    resArr = np.zeros(searchArr.size)
    k = 0
    for step in searchArr:
        resArr[k] = residual(x0 + step*delFF).norm()
        k +=1
    # print(resArr)
    optimalStep = searchArr[resArr.argmin()]
    print('Optimal factor from line search is ', optimalStep)
    return (x0 + optimalStep* delFF)

searchArrFine = np.arange(-0.1,0.11,0.01)
def lineSearchFine(x0,arr):
    delFF = (weighted2ff(flowDict=x0.flowDict,arr=arr).view4d())
    resArr = np.zeros(searchArrFine.size)
    k = 0
    for step in searchArrFine:
        resArr[k] = residual(x0 + step*delFF).norm()
            
    print(resArr)
    print(searchArrFine)
    optimalStep = searchArrFine[resArr.argmin()]
    print('Optimal factor from line search is ', optimalStep)
    return (x0 + optimalStep* delFF)
    
        

In [ ]:
f = h5py.File("", 'r')
self.u = np.array(f['data']['u'])
self.x = np.array(f['geom']['x'])
self.y = np.array(f['geom']['y'])
self.z = np.array(f['geom']['z'])

In [25]:
vf,pf,paramDict = mapData2ff(eps=0.01,g=0.8,theta=0,Re=100.)
x0 = vf.appendField(pf)
print(residual(x0).norm())
#x0 = x0.slice(L=x0.nx//2+2)
#print(residual(x0).slice(L=x0.nx//2-2).norm(), x0.shape)
x0.flowDict['eps'] = 0.01
#x0[:] = 0.; x0[0,x0.nx//2,x0.nz//2,0] = 1.-x0.y**2
print(residual(x0).norm(), x0.getScalar(nd=2).norm(), x0.ddz().norm())

2.90787911136e-13
2.90787911136e-13 1.41853975808e-18 0.0


In [26]:
L = x0.nx//2
Lnew = 2*L
state = x0.slice(L=Lnew).view4d().copyArray()
dx = x0.ddx().slice(L=Lnew).view4d().copyArray()
dy = x0.ddy().slice(L=Lnew).view4d().copyArray()

Phys = np.fft.ifftn(np.fft.ifftshift(state,axes=1),axes=[1])
dxPhys = np.fft.ifftn(np.fft.ifftshift(dx,axes=1),axes=[1])
dyPhys = np.fft.ifftn(np.fft.ifftshift(dy,axes=1),axes=[1])


In [27]:
tempDict = x0.flowDict.copy()
tempDict['L'] = Lnew
SpecArr = np.fft.fftshift(np.fft.fftn(Phys,axes=[1]),axes=1)
Spec = flowFieldWavy(flowDict=tempDict, arr = SpecArr.reshape(SpecArr.size))

dxSpecArr = np.fft.fftshift(np.fft.fftn(dxPhys,axes=[1]),axes=1)
dxSpec = flowFieldWavy(flowDict=tempDict, arr = dxSpecArr.reshape(dxSpecArr.size))

dySpecArr = np.fft.fftshift(np.fft.fftn(dyPhys,axes=[1]),axes=1)
dySpec = flowFieldWavy(flowDict=tempDict, arr = dySpecArr.reshape(dySpecArr.size))


print((Spec.slice(L=L)-x0).norm())
print((dxSpec.slice(L=L)-x0.ddx()).norm())
print((dySpec.slice(L=L)-x0.ddy()).norm())

2.86945998043e-15
1.11982661133e-15
4.76248945806e-15


In [34]:
convPhys = np.zeros(SpecArr.shape,dtype=np.complex)
convPhys[:,:,:,0] = Phys[:,:,:,0]*dxPhys[:,:,:,0] + Phys[:,:,:,1]*dyPhys[:,:,:,0]
convPhys[:,:,:,1] = Phys[:,:,:,0]*dxPhys[:,:,:,1] + Phys[:,:,:,1]*dyPhys[:,:,:,1]

convSpecArr = np.fft.fftshift(np.fft.fftn(convPhys,axes=[1]),axes=1)
convSpec = flowFieldWavy(flowDict=tempDict, arr = convSpecArr.reshape(convSpecArr.size))

x0conv = x0.convNL()

In [72]:
kArr = np.arange(kArr[minArg],kArr[minArg+1],(kArr[minArg+1]-kArr[minArg])/20.)
#kArr = np.arange(32.,34.,0.1)
normArr = np.zeros(kArr.size)
j = 0
for k in kArr:
    normArr[j]=(vecnorm2((convSpec*k-x0conv.slice(L=Lnew)),x0.N))
    j+= 1
print(normArr,'**************')
minArg = np.argmin(normArr)
print(minArg, kArr[minArg], normArr[0], normArr[minArg])


[ 0.00213554  0.00213554  0.00213554  0.00213554  0.00213554  0.00213554
  0.00213554  0.00213554  0.00213554  0.00213554  0.00213554  0.00213554
  0.00213554  0.00213554  0.00213554  0.00213554  0.00213554  0.00213554
  0.00213554  0.00213554] **************
0 33.0 0.00213554091562 0.00213554091562


In [50]:
x0.flowDict['eps'] = 0.0
#x0 = x00.copy()
#x0[:]=0.; x0[0,x0.nx//2,x0.nz//2,0] = 1.-x0.y**2
startTime = time.time()
r0 = residual(x0)
r0norm = r0.norm()
print('$||r_0|| $',r0norm)
for iter in range(5):
    if r0norm < 1.0e-15: print('Norm below tolerance, exiting....',r0norm); break
    jcbn = lambda arr: jacobian(x0,r0,arr)
    A = LinearOperator((x0.size,x0.size), matvec=jcbn,dtype=np.complex)
    delX,flag = lgmres(A,-r0.weighted(),tol=1.0e-15,maxiter=5,inner_m=1000,outer_k=3)
    if flag: print('LGMRES did not converge...................................')
    print('||Ax-b||/||b||:',norm(A.matvec(delX)+r0.weighted(),2)/r0norm)
    print('||del X||:',norm(delX,2))
    x0[:] = lineSearch(x0,delX)
    r0 = residual(x0)
    print('New and old norms respectively:',r0.norm(), r0norm)
    r0norm = r0.norm()
    
print('Done........................................')    
print('Time elapsed:',int(time.time()-startTime))

$||r_0|| $ 6.28393362528


KeyboardInterrupt: 

In [53]:
x = x0.view(flowFieldRiblet)
print(type(x))
print((x.ddx2()-x0.ddX2()).norm())

<class 'flowFieldWavy.flowFieldRiblet'>
0.0


In [ ]:
r0 = residual(x0)
jcbn = lambda arr: jacobian(x0,r0,arr)
A = LinearOperator((x0.size,x0.size), matvec=jcbn,dtype=np.complex)
r0norm = r0.norm()
print('residual norm before correction:',r0norm)
startTime = time.time()
delX,flag = gmres(A,-r0.weighted(),tol=1.0e-6*r0norm,maxiter=50,restart=30)
if flag: print('GMRES did not converge...................................')
print('Norm of Ax-b:',norm(A.matvec(delX)+r0.weighted(),2))
print('Residual norm after correction:', residual(x0+weighted2ff(flowDict=x0.flowDict,arr=delX)).norm())
print('Norm of correction:',norm(delX,2))
x0[:] = lineSearch(x0,delX)
print('New and old norms respectively:',residual(x0).norm(), r0.norm())
print('Time elapsed:',time.time()-startTime)

### Generate numpy arrays for flowfield solutions (velocity, pressure) from Gibson's ChannelFlow 

In [4]:
f = np.genfromtxt('eq1.asc',dtype=np.complex)
f_p = np.genfromtxt('peq1.asc',dtype=np.complex)
   
x = f[:,3];x_p = f_p[:,3]

# From the database at channelflow.org
Re=4.0e2
a=1.14
b=2.5

nx = 48
nz = 25
L = 24
M = 24
N = 35

# In input file, ordering of modes is different from what I would like it to be.
# It goes, 0,1,..,24,-23,-22,....,-1 in 'x'
# I prefer, -23,-22,..,-1,0,1,...,22,23
# In 'z', it's just 0,..,24 
#     Negative wavenumbers are not included due to symmetry.

# First, getting rid of mode 24 in both x and z 
modes = x.reshape((nx,nz,3,N))
modes = sp.append(modes,x_p.reshape((nx,nz,1,N)),axis=2)
modes = sp.delete(modes,L,0)
modes = sp.delete(modes,M,1)
nx = nx-1 
nz = nz-1
L = L-1
M = M-1

# Now, reordering streamwise wavenumbers
modes = sp.append(modes[L+1:,:,:,:],modes[:L+1,:,:,:],0)
del x,x_p,f,f_p

# Projecting Cheb coefficients onto Cheb collocation nodes
for kx in range(0,nx):
    for kz in range(0,nz):
        for nd in range(0,4):
            modes[kx,kz,nd,:] = chebcoll_vec(modes[kx,kz,nd,:])

# Adding modes for negative spanwise wavenumbers. 
modes = sp.append(modes[::-1,:0:-1].conjugate(),modes,axis=1)
nz = nx

# Deleting pressure from state-vector:
p = modes[:,:,3]
modes = sp.delete(modes,3,2)

vecsize = nx*nz*N
# Finally, reshaping to get a state-vector containing u and w
v = modes.reshape(3*vecsize)
p = p.reshape(vecsize)
del modes
#print(modes[L,M,0])


In [5]:
tempDict = getDefaultDict()
L = 23; M = 23; N=35
tempDict.update({'K':0, 'L':L,'M':M,'N':N, 'eps':0.0, 'alpha':1.14,'beta':2.5,'nd':3,'isPois':0, 'Re':400.})
vf = flowFieldRiblet(flowDict=tempDict, arr=v).view4d()
vf[0,L,M,0] += vf.y
tempDict['nd']=1
pf = flowFieldRiblet(flowDict=tempDict, arr=p).view4d()
x0 = vf.appendField(pf)


#x0 = x0.slice(L=3,M=5,N=15)

In [66]:
res = residual(x0)


In [67]:
l=23;m=23
ddz = x0.ddz(); ddZ = x0.ddZ()
for l in range(24):
    print(vecnorm2(res[0,l,:,3],res.N), vecnorm2(ddz[0,l]-ddZ[0,l],x0.N) )

0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
2.00789873167e-19 0.0
1.65731932559e-18 0.0
7.41178185407e-18 0.0
2.39149411705e-17 0.0
1.27990697868e-16 0.0
2.48790696632e-16 0.0
8.28241775515e-16 0.0
2.05247139553e-15 0.0
1.11984562124e-14 0.0
2.91352806371e-14 0.0
1.96116326074e-13 0.0
8.07986020351e-13 0.0
4.98314502809e-12 0.0
1.68395428682e-11 0.0
8.18431896059e-11 0.0
3.50263399547e-11 0.0


In [40]:
x0[:] = 0.
x0[0,23,23,0] += x0.y
x0.norm()

0.15512504992764775

In [150]:
x0 = x0.slice(L=5,M=9,N=35)
residual(x0).norm()

0.00038552857433352177

In [106]:
x0[:] = 0.
x0[0,23,23,0] = x0.y
residual(x0).norm()

2.4185396384947908e-15

In [107]:
x0.slice(nd=[0,1,2]).convNL().norm()

0.0

# -------------------------------------------------

In [ ]:
#from seprn import seprn
#%load_ext autoreload
#%autoreload 2

xs = np.zeros(vArr.shape,dtype=np.float)
xr = xs.copy(); ybub=xs.copy(); sflag = np.zeros(vArr.shape,dtype=np.int)

for ke in range(xs.shape[0]):
    for kg in range(xs.shape[1]):
        for kr in range(xs.shape[2]):
            x = vArr[ke,kg,kr].field.addBase().appendField(pArr[ke,kg,kr].field).view1d()
            N = x.N; n = x.flowDict['L']; a = x.flowDict['alpha']; b=x.flowDict['beta'];Re=x.flowDict['Re']; eps=x.flowDict['eps']
            x = x.copyArray()
            sf,sp,rp,yb = seprn(x,N,n,eps,a,b,Re)
            xs[ke,kg,kr] = sp; xr[ke,kg,kr] = rp; ybub[ke,kg,kr] = yb; sflag[ke,kg,kr] = sf


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
epsArr = vArr[:,:,0].getProperty('eps')
gArr = vArr[:,:,0].getProperty('g')

plt.figure()
plt.contourf(2.*gArr,2.*epsArr,xs[:,:,10])
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.title('Location of separation point, $Re_{\\tau}=4.5$')
plt.contourf(2.*gArr,2.*epsArr,xs[:,:,0],levels=np.arange(0.,0.55,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xSepReT4.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Location of separation point, $Re_{\\tau}=8$')
plt.contourf(2.*gArr,2.*epsArr,xs[:,:,5],levels=np.arange(0.,0.55,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xSepReT8.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Location of separation point, $Re_{\\tau}=14.1$')
plt.contourf(2.*gArr,2.*epsArr,xs[:,:,10],levels=np.arange(0.,0.55,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xSepReT14.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Location of separation point, $Re_{\\tau}=45$')
plt.contourf(2.*gArr,2.*epsArr,xs[:,:,20],levels=np.arange(0.,0.55,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xSepReT45.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Location of Reattachment point, $Re_{\\tau}=4.5$')
plt.contourf(2.*gArr,2.*epsArr,xr[:,:,0],levels=np.arange(0.5,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xReatReT4.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Location of Reattachment point, $Re_{\\tau}=8$')
plt.contourf(2.*gArr,2.*epsArr,xr[:,:,5],levels=np.arange(0.5,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xReatReT8.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Location of Reattachment point, $Re_{\\tau}=14.1$')
plt.contourf(2.*gArr,2.*epsArr,xr[:,:,10],levels=np.arange(0.5,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xReatReT14.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Location of Reattachment point, $Re_{\\tau}=45$')
plt.contourf(2.*gArr,2.*epsArr,xr[:,:,20],levels=np.arange(0.5,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/xReatReT45.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Separation bubble height, $Re_{\\tau}=4.5$')
plt.contourf(2.*gArr,2.*epsArr,ybub[:,:,0],levels=np.arange(0.,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/ybubReT4.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Separation bubble height, $Re_{\\tau}=8$')
plt.contourf(2.*gArr,2.*epsArr,ybub[:,:,5],levels=np.arange(0.,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/ybubReT8.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Separation bubble height, $Re_{\\tau}=14.1$')
plt.contourf(2.*gArr,2.*epsArr,ybub[:,:,10],levels=np.arange(0.,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/ybubReT14.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('Separation bubble height, $Re_{\\tau}=45$')
plt.contourf(2.*gArr,2.*epsArr,xr[:,:,20],levels=np.arange(0.,1.05,0.05))
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.colorbar()
plt.savefig('./figures/ybubReT45.pdf',format='pdf')
plt.show()


In [ ]:
#fluxArr = vArr.getProperty('flux',withBase=True)

import matplotlib.lines as mlines
eOff = 7
plt.figure()
ax = plt.gca()
plt.title ('Contours of Volume flux and bubble height, $Re_{\\tau}=4.5$')
plt.xlabel ('Slope');  plt.ylabel ('Amplitude (log scale)')
ax.set_yscale('log')
CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],fluxArr[eOff:,:,0],levels=[0.4,0.45,0.5,0.55,0.6,0.65],colors = 'k')
plt.clabel(CF)
plt.draw()

CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],ybub[eOff:,:,0],levels=[0.1,0.3,0.5,0.7],colors = 'k',linestyles = 'dashed')
plt.clabel(CF)
plt.draw()

black_line = mlines.Line2D([], [], color='k', label='Vflux',linewidth = 1.5)
dashed_line = mlines.Line2D([], [], color='k', label='yBub',linewidth = 1,linestyle = '--')
plt.legend(handles=[dashed_line,black_line],framealpha = 0.3,loc = 'lower left', ncol = 2)#bbox_to_anchor=(0.24, 0.20),)
plt.savefig('./figures/vFluxYbubReT4.pdf',format='pdf')
plt.show()



plt.figure()
ax = plt.gca()
plt.title ('Contours of Volume flux and bubble height, $Re_{\\tau}=8$')
plt.xlabel ('Slope');  plt.ylabel ('Amplitude (log scale)')
ax.set_yscale('log')
CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],fluxArr[eOff:,:,5],levels=[0.4,0.45,0.5,0.55,0.6,0.65],colors = 'k')
plt.clabel(CF)
plt.draw()

CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],ybub[eOff:,:,5],levels=[0.1,0.3,0.5,0.7],colors = 'k',linestyles = 'dashed')
plt.clabel(CF)
plt.draw()

black_line = mlines.Line2D([], [], color='k', label='Vflux',linewidth = 1.5)
dashed_line = mlines.Line2D([], [], color='k', label='yBub',linewidth = 1,linestyle = '--')
plt.legend(handles=[dashed_line,black_line],framealpha = 0.3,loc = 'lower left', ncol = 2)#bbox_to_anchor=(0.24, 0.20),)
plt.savefig('./figures/vFluxYbubReT8.pdf',format='pdf')
plt.show()



plt.figure()
ax = plt.gca()
plt.title ('Contours of Volume flux and bubble height, $Re_{\\tau}=14$')
plt.xlabel ('Slope');  plt.ylabel ('Amplitude (log scale)')
ax.set_yscale('log')
CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],fluxArr[eOff:,:,10],levels=[0.4,0.45,0.5,0.55,0.6,0.65],colors = 'k')
plt.clabel(CF)
plt.draw()

CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],ybub[eOff:,:,10],levels=[0.1,0.3,0.5,0.7],colors = 'k',linestyles = 'dashed')
plt.clabel(CF)
plt.draw()

black_line = mlines.Line2D([], [], color='k', label='Vflux',linewidth = 1.5)
dashed_line = mlines.Line2D([], [], color='k', label='yBub',linewidth = 1,linestyle = '--')
plt.legend(handles=[dashed_line,black_line],framealpha = 0.3,loc = 'lower left', ncol = 2)#bbox_to_anchor=(0.24, 0.20),)
plt.savefig('./figures/vFluxYbubReT14.pdf',format='pdf')
plt.show()



plt.figure()
ax = plt.gca()
plt.title ('Contours of Volume flux and bubble height, $Re_{\\tau}=45$')
plt.xlabel ('Slope');  plt.ylabel ('Amplitude (log scale)')
ax.set_yscale('log')
CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],fluxArr[eOff:,:,20],levels=[0.4,0.45,0.5,0.55,0.6,0.65],colors = 'k')
plt.clabel(CF)
plt.draw()

CF = plt.contour(2.*gArr[eOff:],2.*epsArr[eOff:],ybub[eOff:,:,20],levels=[0.1,0.3,0.5,0.7],colors = 'k',linestyles = 'dashed')
plt.clabel(CF)
plt.draw()

black_line = mlines.Line2D([], [], color='k', label='Vflux',linewidth = 1.5)
dashed_line = mlines.Line2D([], [], color='k', label='yBub',linewidth = 1,linestyle = '--')
plt.legend(handles=[dashed_line,black_line],framealpha = 0.3,loc = 'lower left', ncol = 2)#bbox_to_anchor=(0.24, 0.20),)
plt.savefig('./figures/vFluxYbubReT45.pdf',format='pdf')
plt.show()





In [ ]:
#ReTauArr = vArr.getProperty('ReTau')
ReArr = vArr.getProperty('Re')
#fluxArr = vArr.getProperty('flux',withBase=True)
uBulk = fluxArr; ReFlat = ReArr
print(ReFlat.shape, uBulk.shape, type(ReFlat), type(uBulk))
fricFac = 16./ReFlat/uBulk/uBulk
ReWavy = uBulk*ReFlat

In [ ]:
epsArr = vArr.getProperty('eps'); gArr = vArr.getProperty('g')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
phiArr = ReWavy*fricFac
keArr = np.asarray([6,17,21]);kgArr = np.asarray([1,11,22]); nRe=ReWavy.shape[2]
print(epsArr[keArr,0,0])
print(gArr[0,kgArr,0])

markers = ['*','s','^']

kg=kgArr[0]
fig = plt.figure(figsize=(7,5))
ax0 = fig.add_subplot(111)
plt.xlabel('Re',fontsize=25); plt.ylabel('$\\phi$',fontsize=35)
plt.ylim([23.,30.])
plt.xlim([-100,700])

ax4,=ax0.plot(ReWavy[keArr[0],kg,:], 24.*np.ones(nRe),linestyle='--',color='k')
plt.savefig('./figures/phiRe0.pdf',format='pdf')

ax1=ax0.scatter(ReWavy[keArr[0],kg,:], phiArr[keArr[0],kg,:], marker=markers[0],color='b',s=40)
plt.savefig('./figures/phiRe11.pdf',format='pdf')

ax2=ax0.scatter(ReWavy[keArr[1],kg,:], phiArr[keArr[1],kg,:], marker=markers[1],color='c',s=40)
plt.savefig('./figures/phiRe12.pdf',format='pdf')

ax3=ax0.scatter(ReWavy[keArr[2],kg,:], phiArr[keArr[2],kg,:], marker=markers[2],color='r',s=40)
plt.savefig('./figures/phiRe13.pdf',format='pdf')


kg=kgArr[1]
fig = plt.figure(figsize=(7,5))
ax0 = fig.add_subplot(111)
plt.xlabel('Re',fontsize=25); plt.ylabel('$\\phi$',fontsize=35)
plt.ylim([23.,30.])

ax4,=ax0.plot(ReWavy[keArr[0],kg,:], 24.*np.ones(nRe),linestyle='--',color='k')
plt.savefig('./figures/phiRe0.pdf',format='pdf')

ax1=ax0.scatter(ReWavy[keArr[0],kg,:], phiArr[keArr[0],kg,:], marker=markers[0],color='b',s=40)
plt.savefig('./figures/phiRe21.pdf',format='pdf')

ax2=ax0.scatter(ReWavy[keArr[1],kg,:], phiArr[keArr[1],kg,:], marker=markers[1],color='c',s=40)
plt.savefig('./figures/phiRe22.pdf',format='pdf')

ax3=ax0.scatter(ReWavy[keArr[2],kg,:], phiArr[keArr[2],kg,:], marker=markers[2],color='r',s=40)
plt.savefig('./figures/phiRe23.pdf',format='pdf')


kg=kgArr[2]
fig = plt.figure(figsize=(7,5))
ax0 = fig.add_subplot(111)
plt.xlabel('Re',fontsize=25); plt.ylabel('$\\phi$',fontsize=35)
plt.ylim([23.,30.])

ax4,=ax0.plot(ReWavy[keArr[0],kg,:], 24.*np.ones(nRe),linestyle='--',color='k')
#plt.savefig('./figures/phiRe0.pdf',format='pdf')

ax1=ax0.scatter(ReWavy[keArr[0],kg,:], phiArr[keArr[0],kg,:], marker=markers[0],color='b',s=40)
plt.savefig('./figures/phiRe31.pdf',format='pdf')

ax2=ax0.scatter(ReWavy[keArr[1],kg,:], phiArr[keArr[1],kg,:], marker=markers[1],color='c',s=40)
plt.savefig('./figures/phiRe32.pdf',format='pdf')

ax3=ax0.scatter(ReWavy[keArr[2],kg,:], phiArr[keArr[2],kg,:], marker=markers[2],color='r',s=40)
plt.savefig('./figures/phiRe33.pdf',format='pdf')

plt.show()

kg = kgArr[0]
fig = plt.figure(figsize=(7,5))
ax0 = fig.add_subplot(111)
plt.xlabel('Re',fontsize=25); plt.ylabel('$\\phi$',fontsize=35)

ax3=ax0.scatter(ReWavy[keArr[2],kg,:], phiArr[keArr[2],kg,:], marker=markers[2],color='r')
phiMean = 0.5*(np.min(phiArr[keArr[2],kg,:])+np.max(phiArr[keArr[2],kg,:]))
phiDev = np.max(phiArr[keArr[2],kg,:]) - phiMean
print('phiMean, phiDev:',phiMean,phiDev)
ax4=ax0.plot(ReWavy[keArr[2],kg,:],phiMean*np.ones(nRe), color='r')


plt.annotate('$\\bar{\\phi}$',xy = (0.05,0.5),xycoords='axes fraction', 
             xytext=(0.05,0.5),textcoords='axes fraction',
             arrowprops=dict(facecolor='white',shrink=0.000,width=0.0),
            horizontalalignment='left', verticalalignment='top',fontsize=16)

plt.show()



In [ ]:
np.log10(0.03)

In [ ]:
kg=kgArr[0]
fig = plt.figure(figsize=(7,5))
ax0 = fig.add_subplot(111)
plt.xlabel('Re',fontsize=25); plt.ylabel('$\\phi$',fontsize=35)
plt.ylim([23.,30.])
ax3=ax0.scatter(ReWavy[keArr[2],kg,:], phiArr[keArr[2],kg,:], marker=markers[2],color='r',s=40)
plt.savefig('./figures/phiRe40.pdf',format='pdf')
plt.show()

fig = plt.figure(figsize=(7,5))
ax0 = fig.add_subplot(111)
plt.xlabel('Re',fontsize=25); plt.ylabel('$\\phi$',fontsize=35)
ax3=ax0.scatter(ReWavy[keArr[2],kg,:], phiArr[keArr[2],kg,:], marker=markers[2],color='r',s=40)
plt.savefig('./figures/phiRe41.pdf',format='pdf')

phiMean = 0.5*(np.min(phiArr[keArr[2],kg,:])+np.max(phiArr[keArr[2],kg,:]))
phiDev = np.max(phiArr[keArr[2],kg,:]) - phiMean
print('phiMean, phiDev:',phiMean,phiDev)
ax4=ax0.plot(ReWavy[keArr[2],kg,:],phiMean*np.ones(nRe), color='r')
plt.text(-35,25.6,'$\\bar{\\phi}$',fontsize=35,color='blue')
plt.savefig('./figures/phiRe42.pdf',format='pdf')

plt.annotate("",xy=(600,26.22),xycoords='data',
            xytext=(600,25.58),textcoords='data',
             arrowprops={'arrowstyle':'<->'})
plt.text(530,25.9,"$\\phi'$",fontsize=35,color='blue')
plt.savefig('./figures/phiRe43.pdf',format='pdf')
plt.show()


In [ ]:
phiMean = 0.5*(np.min(phiArr,axis=2)+np.max(phiArr,axis=2))
phiDev = 0.5*(np.max(phiArr,axis=2)-np.min(phiArr,axis=2))

## Curve fitting

- We're looking for a relation that goes as $\lambda = a Re^{-n}$, where $a$ and $n$ are functions of $g,\epsilon$. 
- From earlier work by Adrien, we know $n \approx 1$ for $\epsilon$ holds for $4\epsilon$ not exceeding 15%.
- We just need to verify if $\lambda Re = const.$ holds up, and if it does, how we need to see how the constant varies with $g,\epsilon$.

### FIrst, checking if the inverse law holds

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline
#coloring = np.random.random(32).reshape((8,4))
#for k in range(0,8):
#    plt.scatter(np.log10(ReWavy[3*k,3*k]),10.**(np.log10(ReWavy[3*k,3*k])+ np.log10(fricFac[3*k,3*k])), c=coloring[k-3])
#plt.ylim([-0.006,0.])
#plt.show()

#theta='0'
#pp = PdfPages('InverseLaw'+theta+'.pdf')

#prod = fricFac*ReWavy         # This must be a constant for different ReWavy. 
prod = phiArr
# To see if it is indeed constant, I can look at the standard deviation at any g,eps:
#prod = prod[:,:,2:]

#stdDev = np.std(prod,axis=2)  # Computing standard deviation for lambda*Re at a constant g,eps, but varying Re
stdDev=phiDev

epsArr = vArr.getProperty('eps')
gArr = vArr.getProperty('g')
cutOff = 0

cLimits = np.arange(24., 44., 1.0)
#meanProd = np.mean(prod,axis=2)  # Computing mean for lambda*Re at a constant g,eps, but varying Re
meanProd = phiMean
plt.figure()
#plt.contourf(2.*gArr[:-cutOff,:,0], 4.*epsArr[:-cutOff,:,0], meanProd[:-cutOff],levels=cLimits)
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], meanProd[:], levels=cLimits)
#plt.ylim([24.,44.])
#plt.clim(24.0,28.0)
plt.colorbar()
plt.title('$\\bar{\\phi}$',fontsize=25)
plt.xlabel('Slope',fontsize=15);plt.ylabel('Amplitude',fontsize=15)
#pp.savefig()
plt.savefig('./figures/mean.pdf',format='pdf')
plt.show()

cLimits= np.arange(0.0, 0.3, 0.015)
plt.figure()
plt.title("$\\phi'$",fontsize=25)
#plt.contourf(2.*gArr[:-cutOff,:,0], 4.*epsArr[:-cutOff,:,0], stdDev[:-cutOff], levels=cLimits)
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], stdDev[:], levels=cLimits)
#plt.clim([0.,0.15])
#plt.clim(0.0,0.2)
plt.colorbar()
plt.xlabel('Slope',fontsize=15);plt.ylabel('Amplitude',fontsize=15)
#pp.savefig()
plt.savefig('./figures/std.pdf',format='pdf')
plt.show()


cLimits = np.arange(0.0,0.1,0.1/20)
#cLimits= np.append(cLimits,np.asarray([0.2,0.3,0.4,0.5,0.6,0.7]))
plt.figure()
plt.title("$\\phi'/\\bar{\\phi}$",fontsize=25)
#plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], stdDev[:]/(meanProd[:]-24),levels=cLimits)
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], stdDev[:]/(meanProd[:]),levels=cLimits)
#plt.clim([0.,0.15])
#plt.clim(0.0,0.1)
plt.colorbar()
plt.xlabel('Slope',fontsize=15);plt.ylabel('Amplitude',fontsize=15)
plt.savefig('./figures/stdOverMean.pdf',format='pdf')
plt.show()
#plt.savefig('stdOverMeanT'+theta+'.pdf')

# Mean where stdDev/(mean-24) is small enough (<10%):
#cLimits = np.arange(0., 4., 0.25)
#stdOverMean = stdDev/(meanProd-24.)
stdOverMean = stdDev/(meanProd)
newMean = meanProd.copy()
newMean[stdOverMean>0.1] = None
plt.figure()
#plt.title('(Mean-24) of fricFac*ReWavy where inverse relation holds (10% stdDev)')
plt.title('(Mean) of fricFac*ReWavy where inverse relation holds (10% stdDev)')
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], (newMean[:]-24))
plt.colorbar()
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.savefig('meanInverseLaw'+theta+'.pdf',format='pdf')
plt.show()




In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline
#coloring = np.random.random(32).reshape((8,4))
#for k in range(0,8):
#    plt.scatter(np.log10(ReWavy[3*k,3*k]),10.**(np.log10(ReWavy[3*k,3*k])+ np.log10(fricFac[3*k,3*k])), c=coloring[k-3])
#plt.ylim([-0.006,0.])
#plt.show()

#theta='0'
#pp = PdfPages('InverseLaw'+theta+'.pdf')

prod = fricFac*ReWavy         # This must be a constant for different ReWavy. 
# To see if it is indeed constant, I can look at the standard deviation at any g,eps:
#prod = prod[:,:,2:]
#meanProd = np.mean(prod,axis=2)  # Computing mean for lambda*Re at a constant g,eps, but varying Re
meanProd = 0.5*( prod[:,:,0] + prod[:,:,-1] )

#stdDev = np.std(prod,axis=2)  # Computing standard deviation for lambda*Re at a constant g,eps, but varying Re
stdDev = np.amax( np.abs( prod[:,:,:] -meanProd.reshape((prod.shape[0],prod.shape[1],1))  ) , axis=2 )

epsArr = vArr.getProperty('eps')
gArr = vArr.getProperty('g')
cutOff = 0

cLimits = np.arange(24., 44., 1.0)
plt.figure()
#plt.contourf(2.*gArr[:-cutOff,:,0], 4.*epsArr[:-cutOff,:,0], meanProd[:-cutOff],levels=cLimits)
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], meanProd[:], levels=cLimits)
#plt.ylim([24.,44.])
#plt.clim(24.0,28.0)
plt.colorbar()
plt.title('$\\bar{\\phi}$: Mean of $\\phi(Re)$')
plt.xlabel('Slope');plt.ylabel('Amplitude')
#pp.savefig()
plt.savefig('mean.pdf',format='pdf')
plt.show()

#cLimits= np.arange(0.0, 0.3, 0.05)
plt.figure()
plt.title("$\\phi'$: Deviation of $\\phi(Re)$ from $\\bar{\\phi}$")
#plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], stdDev[:], levels=cLimits)
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], stdDev[:],20)
#plt.clim([0.,0.15])
#plt.clim(0.0,2.0)
plt.colorbar()
plt.xlabel('Slope');plt.ylabel('Amplitude')
#pp.savefig()
plt.savefig('stdT'+theta+'.pdf',format='pdf')
plt.show()


cLimits = np.arange(0.0,0.15,0.1/20)
#cLimits= np.append(cLimits,np.asarray([0.2,0.3,0.4,0.5,0.6,0.7]))
plt.figure()
plt.title("$\\phi'/\\bar{\\phi}$: Relative deviation")
#plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], stdDev[:]/(meanProd[:]-24),levels=cLimits)
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], stdDev[:]/(meanProd[:]),levels=cLimits)
#plt.clim([0.,0.15])
#plt.clim(0.0,0.1)
plt.colorbar()
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.savefig('stdOverMeanT'+theta+'.pdf',format='pdf')
plt.show()
#plt.savefig('stdOverMeanT'+theta+'.pdf')

# Mean where stdDev/(mean-24) is small enough (<10%):
#cLimits = np.arange(0., 4., 0.25)
#stdOverMean = stdDev/(meanProd-24.)
stdOverMean = stdDev/(meanProd)
newMean = meanProd.copy()
newMean[stdOverMean>0.1] = None
plt.figure()
#plt.title('(Mean-24) of fricFac*ReWavy where inverse relation holds (10% stdDev)')
plt.title('(Mean) of fricFac*ReWavy where inverse relation holds (10% stdDev)')
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], (newMean[:]-24))
plt.colorbar()
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.savefig('meanInverseLaw'+theta+'.pdf',format='pdf')
plt.show()


In [ ]:
plt.figure()
lArr = np.random.rand(120).reshape(30,4)
symArr = ["o","v","s","^","*","x","+"]
symArr.extend(symArr);symArr.extend(symArr);
printFolder = '/home/sabarish/Dropbox/reports/2015/transfer/figures/'
'''
for k in range(21):
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    plt.scatter(2.*gArr[k,:,0],(meanProd[k,:]-24.), color=lArr[k])
plt.show()
'''
plt.figure()
#ax = plt.subplot(111)
#ax = plt.axes()
plt.annotate('Slope',xy = (0.4,0.7),xycoords='axes fraction', 
             xytext=(0.7,0.1),textcoords='axes fraction',
             arrowprops=dict(facecolor='black',shrink=0.0001,width=0.01),
            horizontalalignment='left', verticalalignment='top',fontsize=12)
keArr=[0,1,2,3,4,6,8,11,14,18,23]
for k in keArr:
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    #plt.scatter(2.*epsArr[:,k,0],(meanProd[:,k]-24.), color=lArr[k])
    plt.plot(2.*epsArr[:,k,0],(meanProd[:,k]))#, marker=symArr[k])
plt.title('$\\bar{\\phi}$-dependence on Amplitude')
plt.xlabel('Amplitude')
plt.ylabel('$\\bar{\\phi}$',fontsize=12)
plt.ylim([23.,43.])
plt.savefig('./figures/epsDep.pdf',format='pdf')
plt.show()

kgArr=[0,5,10,14,17,19,21,22,23,24,25]
plt.figure()
plt.annotate('Amplitude',xy = (0.5,0.9),xycoords='axes fraction', 
             xytext=(0.5,0.05),textcoords='axes fraction',
             arrowprops=dict(facecolor='black',shrink=0.0001,width=0.01),
            horizontalalignment='left', verticalalignment='top',fontsize=12)
for k in kgArr:
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    #plt.scatter(2.*gArr[k,4:,0],(meanProd[k,4:]-24.), color=lArr[k])
    plt.plot(2.*gArr[k,4:,0],(meanProd[k,4:]))
plt.title('$\\bar{\\phi}$-dependence on Slope')
plt.xlabel('Slope')
plt.ylabel('$\\bar{\\phi}$',fontsize=12)
plt.ylim([23.,43.])
plt.savefig('./figures/gDep.pdf',format='pdf')
plt.show()




plt.figure()
nRe = ReWavy[0,0,:].size

#keArr = np.asarray([3,10,20,3,10,20,3,10,20]);kgArr = np.asarray([0,0,0,10,10,10,20,20,20])
"""
for k in range(keArr.size):
    ke = keArr[k]; kg = kgArr[k]
    plt.figure()
    ax1=plt.scatter(ReWavy[ke,kg,:], ReWavy[ke,kg,:]*fricFac[ke,kg,:])
    ax2,=plt.plot(ReWavy[ke,kg,:], np.mean(ReWavy[ke,kg,:]*fricFac[ke,kg,:])*np.ones(nRe))
    ax3,=plt.plot(ReWavy[ke,kg,:], 24.*np.ones(nRe),linestyle='--')
    plt.legend([ax1,ax2,ax3], ['computed', 'mean','flatWall'], loc=2)
    #plt.xlabel('ReBulk'); plt.ylabel('fricFac*ReBulk')
    plt.xlabel('Re'); plt.ylabel('$\bar{\phi}$')
    plt.ylim([23.,29.])
    #plt.title('fricFac*ReBulk for Amplitude:'+str(round(2.*epsArr[ke,0,0],4))+', Slope:'+str(2.*gArr[0,kg,0]) )
    plt.title('$\bar{\phi}$ for Amplitude:'+str(round(2.*epsArr[ke,0,0],4))+', Slope:'+str(2.*gArr[0,kg,0]) )
    plt.savefig(printFolder+'linReg'+str(k)+'.pdf', format='pdf')
"""
keArr = np.asarray([3,11,20]);kgArr = np.asarray([0,10,20])
markers = ['x','+','^']
for k in range(kgArr.size):
    kg = kgArr[k]
    fig = plt.figure(figsize=(7,5))
    ax0 = fig.add_subplot(111)
    ax1=ax0.scatter(ReWavy[keArr[0],kg,:], ReWavy[keArr[0],kg,:]*fricFac[keArr[0],kg,:], marker=markers[0],color='b')
    ax2=ax0.scatter(ReWavy[keArr[1],kg,:], ReWavy[keArr[1],kg,:]*fricFac[keArr[1],kg,:], marker=markers[1],color='g')
    ax3=ax0.scatter(ReWavy[keArr[2],kg,:], ReWavy[keArr[2],kg,:]*fricFac[keArr[2],kg,:], marker=markers[2],color='r')
    #ax2,=plt.plot(ReWavy[ke,kg,:], np.mean(ReWavy[ke,kg,:]*fricFac[ke,kg,:])*np.ones(nRe))
    ax4,=ax0.plot(ReWavy[keArr[0],kg,:], 24.*np.ones(nRe),linestyle='--',color='k')
    plt.legend([ax1,ax2,ax3,ax4], \
               ['Amp:'+str(round(2.*epsArr[keArr[0],0,0],4)),\
                'Amp:'+str(round(2.*epsArr[keArr[1],0,0],4)),\
                'Amp:'+str(round(2.*epsArr[keArr[2],0,0],4)),\
                'flat'], loc=2)
    #plt.xlabel('ReBulk'); plt.ylabel('fricFac*ReBulk')
    plt.xlabel('Re'); plt.ylabel('phi')
    plt.ylim([23.,30.])
    #plt.title('fricFac*ReBulk for Amplitude:'+str(round(2.*epsArr[ke,0,0],4))+', Slope:'+str(2.*gArr[0,kg,0]) )
    plt.title('$\\bar{\\phi}$ for Slope:'+str(2.*gArr[0,kg,0]) )
    plt.savefig(printFolder+'linReg'+str(k)+'.pdf', format='pdf')




In [ ]:
plt.figure()
plt.annotate('Slope',xy = (0.4,0.7),xycoords='axes fraction', 
             xytext=(0.7,0.1),textcoords='axes fraction',
             arrowprops=dict(facecolor='black',shrink=0.0001,width=0.01),
            horizontalalignment='left', verticalalignment='top',fontsize=12)
keArr=[0,1,2,3,4,6,8,11,14,18,23]
for k in keArr:
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    #plt.scatter(2.*epsArr[:,k,0],(meanProd[:,k]-24.), color=lArr[k])
    plt.plot(2.*epsArr[:,k,0],(meanProd[:,k]))#, marker=symArr[k])
plt.title('$\\bar{\\phi}$-dependence on Amplitude',fontsize=18)
plt.xlabel('Amplitude',fontsize=15)
plt.ylabel('$\\bar{\\phi}$',fontsize=25)
plt.ylim([23.,43.])
plt.savefig('./figures/epsDep.pdf',format='pdf')
plt.show()

kgArr=[0,5,10,14,17,19,21,22,23,24,25]
plt.figure()
plt.annotate('Amplitude',xy = (0.5,0.9),xycoords='axes fraction', 
             xytext=(0.5,0.05),textcoords='axes fraction',
             arrowprops=dict(facecolor='black',shrink=0.0001,width=0.01),
            horizontalalignment='left', verticalalignment='top',fontsize=12)
for k in kgArr:
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    #plt.scatter(2.*gArr[k,4:,0],(meanProd[k,4:]-24.), color=lArr[k])
    plt.plot(2.*gArr[k,4:,0],(meanProd[k,4:]))
plt.title('$\\bar{\\phi}$-dependence on Slope',fontsize=18)
plt.xlabel('Slope',fontsize=15)
plt.ylabel('$\\bar{\\phi}$',fontsize=25)
plt.ylim([23.,43.])
plt.savefig('./figures/gDep.pdf',format='pdf')
plt.show()


In [ ]:
plt.figure()
lArr = np.random.rand(120).reshape(30,4)
symArr = ["o","v","s","^","*","x","+"]
symArr.extend(symArr);symArr.extend(symArr);
printFolder = '/home/sabarish/Dropbox/reports/2015/transfer/figures/'
'''
for k in range(21):
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    plt.scatter(2.*gArr[k,:,0],(meanProd[k,:]-24.), color=lArr[k])
plt.show()
'''
plt.figure()
#ax = plt.subplot(111)
#ax = plt.axes()
plt.annotate('Slope',xy = (0.4,0.7),xycoords='axes fraction', 
             xytext=(0.7,0.1),textcoords='axes fraction',
             arrowprops=dict(facecolor='black',shrink=0.0001,width=0.01),
            horizontalalignment='left', verticalalignment='top',fontsize=12)
keArr=[0,1,2,3,4,6,8,11,14,18,23]
for k in keArr:
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    #plt.scatter(2.*epsArr[:,k,0],(meanProd[:,k]-24.), color=lArr[k])
    plt.plot(2.*epsArr[:,k,0],(meanProd[:,k]))#, marker=symArr[k])
#plt.title('Amplitude depencence of $\\bar{\\phi}$')
plt.xlabel('Amplitude ($2 \\epsilon$)')
plt.ylabel('$\\bar{\\phi}$')
plt.ylim([23.,43.])
plt.savefig(printFolder+'epsDep.pdf',format='pdf')
plt.show()

kgArr=[0,5,10,14,17,19,21,22,23,24,25]
plt.figure()
plt.annotate('Amplitude',xy = (0.5,0.9),xycoords='axes fraction', 
             xytext=(0.5,0.05),textcoords='axes fraction',
             arrowprops=dict(facecolor='black',shrink=0.0001,width=0.01),
            horizontalalignment='left', verticalalignment='top',fontsize=12)
for k in kgArr:
    #plt.scatter(np.log10(2.*gArr[k,:,0]),np.log10(meanProd[k,:]-24.), color=lArr[k])
    #plt.scatter(2.*gArr[k,4:,0],(meanProd[k,4:]-24.), color=lArr[k])
    plt.plot(2.*gArr[k,4:,0],(meanProd[k,4:]))
#plt.title('Slope dependence of $\\bar{\\phi}$')
plt.xlabel('Slope (2g)')
plt.ylabel('$\\bar{\\phi}$')
plt.ylim([23.,43.])
plt.savefig(printFolder+'gDep.pdf',format='pdf')
plt.show()




plt.figure()
nRe = ReWavy[0,0,:].size

#keArr = np.asarray([3,10,20,3,10,20,3,10,20]);kgArr = np.asarray([0,0,0,10,10,10,20,20,20])
"""
for k in range(keArr.size):
    ke = keArr[k]; kg = kgArr[k]
    plt.figure()
    ax1=plt.scatter(ReWavy[ke,kg,:], ReWavy[ke,kg,:]*fricFac[ke,kg,:])
    ax2,=plt.plot(ReWavy[ke,kg,:], np.mean(ReWavy[ke,kg,:]*fricFac[ke,kg,:])*np.ones(nRe))
    ax3,=plt.plot(ReWavy[ke,kg,:], 24.*np.ones(nRe),linestyle='--')
    plt.legend([ax1,ax2,ax3], ['computed', 'mean','flatWall'], loc=2)
    #plt.xlabel('ReBulk'); plt.ylabel('fricFac*ReBulk')
    plt.xlabel('Re'); plt.ylabel('$\bar{\phi}$')
    plt.ylim([23.,29.])
    #plt.title('fricFac*ReBulk for Amplitude:'+str(round(2.*epsArr[ke,0,0],4))+', Slope:'+str(2.*gArr[0,kg,0]) )
    plt.title('$\bar{\phi}$ for Amplitude:'+str(round(2.*epsArr[ke,0,0],4))+', Slope:'+str(2.*gArr[0,kg,0]) )
    plt.savefig(printFolder+'linReg'+str(k)+'.pdf', format='pdf')
"""
keArr = np.asarray([3,11,20]);kgArr = np.asarray([0,10,20])
markers = ['x','+','^']
for k in range(kgArr.size):
    kg = kgArr[k]
    fig = plt.figure(figsize=(7,5))
    ax0 = fig.add_subplot(111)
    ax1=ax0.scatter(ReWavy[keArr[0],kg,:], ReWavy[keArr[0],kg,:]*fricFac[keArr[0],kg,:], marker=markers[0],color='b')
    ax2=ax0.scatter(ReWavy[keArr[1],kg,:], ReWavy[keArr[1],kg,:]*fricFac[keArr[1],kg,:], marker=markers[1],color='g')
    ax3=ax0.scatter(ReWavy[keArr[2],kg,:], ReWavy[keArr[2],kg,:]*fricFac[keArr[2],kg,:], marker=markers[2],color='r')
    #ax2,=plt.plot(ReWavy[ke,kg,:], np.mean(ReWavy[ke,kg,:]*fricFac[ke,kg,:])*np.ones(nRe))
    ax4,=ax0.plot(ReWavy[keArr[0],kg,:], 24.*np.ones(nRe),linestyle='--',color='k')
    plt.legend([ax1,ax2,ax3,ax4], \
               ['Amp:'+str(round(2.*epsArr[keArr[0],0,0],4)),\
                'Amp:'+str(round(2.*epsArr[keArr[1],0,0],4)),\
                'Amp:'+str(round(2.*epsArr[keArr[2],0,0],4)),\
                'flat'], loc=2)
    #plt.xlabel('ReBulk'); plt.ylabel('fricFac*ReBulk')
    plt.xlabel('Re'); plt.ylabel('$\\phi$')
    plt.ylim([23.,30.])
    #plt.title('fricFac*ReBulk for Amplitude:'+str(round(2.*epsArr[ke,0,0],4))+', Slope:'+str(2.*gArr[0,kg,0]) )
    #plt.title('Re-variation of $\\phi$, for Slope:'+str(2.*gArr[0,kg,0]) )
    plt.savefig(printFolder+'linReg'+str(k)+'.pdf', format='pdf')




In [ ]:
from scipy.optimize import leastsq

meanData = meanProd[:, 4:]
epsVec2 = 2.*epsArr[:,0,0]; gVec2 = 2.*gArr[0,4:,0]
eps2d = 2.*epsArr[:,4:,0]; g2d = 2.*gArr[:,4:,0]



eps1d = eps2d.reshape(eps2d.size); g1d = g2d.reshape(g2d.size)
meanData1d = meanData.reshape(meanData.size)

def __residuals(params,meanData1d,eps1d, g1d):
    # Fitting meanData to  (a*tanh(b*g2d)+c)*eps2d +d*eps2d
    # a = params[0]; b=params[1]; c=params[2]
    #residual = 24+(params[0]*np.tanh(params[1]*g1d) + params[2])*eps1d + params[3]*eps1d**2  - meanData1d
    
    # Fitting meanData to 24 + [A*tanh(B*g1d) + 72-A]*eps1d + 144.*eps1d 
    residual = 24+(params[0]*np.tanh(params[1]*g1d) + 72.-params[0])*eps1d + (params[2]*np.tanh(params[3]*g1d) + 144.-params[2])*eps1d**2  - meanData1d
    return residual

#print(meanData.shape, eps2d.shape, g2d.shape)
#test = __residuals([70.,1.,10.], meanData, eps2d, g2d)
#print(type(test),test.shape)
pSol = leastsq(__residuals, np.array([50.,1.,150.,1.]), args=(meanData1d,eps1d,g1d))[0]

plt.figure()
plt.contourf(g2d, eps2d,  meanData, levels=np.arange(23.,43.,1.))
plt.colorbar()
plt.title('$\\bar{\\phi}$')
plt.xlabel('Slope'); plt.ylabel('Amplitude')
#plt.savefig('mean.pdf',format='pdf')
#plt.yscale('log')
plt.show()

plt.figure()
plt.contourf(g2d, eps2d, __residuals(pSol, meanData,eps2d,g2d)+meanData, levels=np.arange(23.,43.,1.0))
plt.colorbar()
plt.title('Model: $(A\\tanh(Bg) +72-A)\epsilon + (C\\tanh(Dg) +144-C)\epsilon^2$')
plt.xlabel('Slope'); plt.ylabel('Amplitude')
#plt.yscale('log')
plt.savefig('./figures/model.pdf', format='pdf')
plt.show()

plt.figure()
plt.contourf(g2d, eps2d, np.abs(__residuals(pSol, meanData,eps2d,g2d)), levels=np.arange(0.,0.5,0.01))
plt.colorbar()
plt.title('Error in $\\bar{\\phi}$')
plt.xlabel('Slope'); plt.ylabel('Amplitude')
#plt.yscale('log')
plt.savefig('./figures/modelDiff.pdf', format='pdf')
plt.show()

plt.figure()
plt.contourf(g2d, eps2d, np.abs(__residuals(pSol, meanData,eps2d,g2d)/meanData), levels=np.arange(0.,0.02,0.001))
plt.colorbar()
plt.title('Relative error in $\\bar{\\phi}$',fontsize=20)
plt.xlabel('Slope',fontsize=15); plt.ylabel('Amplitude',fontsize=15)
#plt.yscale('log')
plt.savefig('./figures/modelRelDiff.pdf', format='pdf')
plt.show()

print('Model coefficients:a=%1.3f,b=%1.3f,c=%1.3f,d=%1.3f'%(pSol[0],pSol[1],pSol[2], pSol[3]) )



In [ ]:
# Printing to ascii for 3-d visualization in tec360:
fricFac2d = fricFac[:,:]; ReWavy2d = ReWavy[:,:]
ReWavy1d = ReWavy2d.reshape(ReWavy2d.size); fricFac1d = fricFac2d.reshape(fricFac2d.size)
eps2d = 2.*epsArr; g2d = 2.*gArr
eps1d = eps2d.reshape(eps2d.size); g1d = g2d.reshape(g2d.size)
modelMean = ((pSol[0]*np.tanh(pSol[1]*g1d) + pSol[2])*eps1d + pSol[3]*eps1d**2)+24.
modelFricFac = modelMean/ReWavy1d
#modelFricFac = ((.*np.tanh(0.6*g1d) + 10.)*eps1d + 136.*eps1d**2)+24.
fricFacDiff = np.abs(modelFricFac-fricFac1d)
fricFacRelDiff = np.abs(modelFricFac-fricFac1d)/fricFac1d

modelDiff = np.abs(modelFricFac - fricFac1d*ReWavy1d)
modelDiffRelative = modelDiff/(fricFac1d*ReWavy1d-24.)

printArr = np.concatenate((ReWavy1d, g1d, eps1d, fricFac1d, modelFricFac, fricFacDiff, fricFacRelDiff), axis=0)
fName='./figures/modelFFv1'
variables = 'VARIABLES = "Re(bulk)", "Slope", "Amplitude", "fricFac","fricFacModel", "fricFacError", "fricFacRelError"\n'
zone = 'ZONE T="", I='+str(ReWavy2d.shape[2])+', J='+str(ReWavy2d.shape[1])+', K='+str(ReWavy2d.shape[0])+', DATAPACKING=POINT \n'
title = 'TITLE= "Product of friction factor and Reynolds number: computed and modeled"\n'
hdr = title+variables+zone

print(type(printArr),printArr.shape)
np.savetxt(fName+'.dat',printArr.reshape(7,printArr.size//7).T, header=hdr,comments='')
print('Printed data to file %s.dat'%fName)



In [ ]:
epsVec = epsArr[:-5,0,0]
plt.plot(epsVec,aArr)
plt.title('eps vs a: in mean = a*tanh(b*g) +c')
plt.xlabel('eps'); plt.ylabel('a');plt.show()


plt.figure()
plt.plot(epsVec,bArr)
plt.title('eps vs b in mean = a*tanh(b*g) +c')
plt.xlabel('eps'); plt.ylabel('b');plt.show()

plt.figure()
plt.plot(epsVec,cArr)
plt.title('eps vs c in mean = a*tanh(b*g) +c')
plt.xlabel('eps'); plt.ylabel('c');plt.show()


In [ ]:
linFun = lambda xData,k: k*xData
sqrtFun = lambda xData,k: k*np.sqrt(xData)

aCoeff,pcov = curve_fit(linFun,epsVec, aArr, p0=100.)
bCoeff,pcov = curve_fit(sqrtFun, epsVec,bArr)
cCoeff,pcov = curve_fit(linFun, epsVec, cArr,p0=20.)

print(aCoeff, bCoeff,cCoeff)

In [ ]:
epsArr2 = epsArr[:,:,0]; gArr2 = gArr[:,:,0]
curveFitted = 24.+ cCoeff*epsArr2 + aCoeff*epsArr2*np.tanh(bCoeff*np.sqrt(epsArr2)*2.*gArr2)
originalData = meanProd[:-5]

plt.contourf(2.*gArr2, 4.*epsArr2,originalData-24.)
plt.title('Original data (mean-24)')
plt.xlabel('slope=2*g'); plt.ylabel('crest-trough amplitude = 4*eps')
plt.colorbar(); plt.show()


plt.figure()
plt.contourf(2.*gArr2, 4.*epsArr2, curveFitted-24.)
plt.title('Curve fitted data (mean-24)')
plt.xlabel('slope=2*g'); plt.ylabel('crest-trough amplitude = 4*eps')
plt.colorbar(); plt.show()

plt.figure()
plt.contourf(2.*gArr2, 2.*epsArr2, np.abs(curveFitted-originalData))
plt.title('Error')
plt.xlabel('Slope'); plt.ylabel('Amplitude')
plt.colorbar(); plt.show()

plt.figure()
plt.contourf(2.*gArr2, 2.*epsArr2, np.abs(curveFitted-originalData)/originalData, levels = np.arange(0.,0.4,0.04))
plt.clim([0.,0.2])
plt.xlabel('Slope'); plt.ylabel('Amplitude')
plt.title('Relative Error')
plt.colorbar(); plt.show()


## FFT and Convolution of spectral coefficients for 1-D arrays


In [188]:
import scipy.signal as sig
def fftMan(y,x): # Manual DFT
    if y[-1]==y[0]: _y = y[:-1]; _x=x[:-1]
    else: _y = y; _x = x
    global alpha
    N = _y.size//2
    coeffs = np.zeros(2*N+1, dtype=np.complex)
    for k in range(-N,N+1):
        coeffs[N+k] = np.dot( _y,np.exp(-1.j*k*alpha*_x) )
    
    coeffs = coeffs/_y.size
    
    if norm(np.imag(y)) == 0.: return np.real(coeffs)
    return coeffs
    
def ifftMan(y,x): # Manual inverseDFT
    assert y.size//2 != 0.
    _y = y
        
    global alpha
    N = _y.size//2
    phys = np.zeros(2*N,dtype=np.complex)
    for k in range(-N,N+1):
        phys += _y[N+k]*np.exp(1.j*k*alpha*x) 
    
    return np.real(phys)

def convMan(y1,y2): # convolution: pad zeros on both sides, do convolution, return middle part
    # I'm supposing the convolution is on the spectral coeffs
    N = y1.size//2
    zArr = np.zeros(N,dtype=np.complex)
    
    res = np.zeros(y1.size+2*N, dtype=np.complex)
    
    for k1 in range(-N,N+1):
        for k2 in range(-N,N+1):
            res[2*N+k1+k2] += y1[N+k1] * y2[N+k2]
    
    return res[N:N+y1.size]

def npfft(x):
    assert x.size//2 != 0., "Numpy's fft only works well on [0,2*pi), i.e., when 2*pi is not included"
    c = np.fft.fftshift(np.fft.fft(x))/x.size
    c = np.append(c,np.conj(c[0]))
    if norm(np.imag(c),2)/c.size<= 1.0e-14: return np.real(c)
    return c

def npifft(x):
    assert x.size//2 != 0., "Coefficient vector must have modes -N through +N, including +N"
    c = np.fft.ifft(np.fft.ifftshift(  x[:-1]*(x.size-1)    ))
    assert norm(np.imag(c),2)/c.size<=1.0e-14, "I only use real fields, so ifft should not have any imaginary parts"
    return np.real(c)

def npconvolve(x,y):
    return np.convolve(x,y,mode='same')

def isZero(x):
    return not (np.abs(x)>1.0e-14).any()



In [231]:
# Testing convolution theorem: convolve(F(y1),F(y2)) = F(y1*y2)
N = 5; A = 2.; alpha = 3.; M = -1
x = (np.arange(-N,N)+N)/N*np.pi/alpha # x goes from -pi/alpha to pi/alpha (pi/alpha not included)
y1 = A*np.cos(M*alpha*x)
y2 = 2.*A*np.cos((M+1)*alpha*x)

c1 = npfft(y1)
c2 = npfft(y2)

#print(npfft(y1))
print("Does npfft comply with convolution theorem, convolution in spectral?:", isZero(npconvolve(npfft(y1),npfft(y2))-npfft(y1*y2))                  )
print("Does npfft match fftMan?:", isZero(fftMan(y1,x)-npfft(y1))                  )
print("Is ifftMan consistent with fftMan?:", isZero( ifftMan(fftMan(y1,x),x)-y1  ) )


Does npfft comply with convolution theorem, convolution in spectral?: True
Does npfft match fftMan?: True
Is ifftMan consistent with fftMan?: True


False

## Ensure npfft and npifft (both custom functions) are consistent with each other


In [116]:
# First, showing that numpy's routines are self-consistent
print("Are numpy's fft and ifft self-consistent?:",isZero(np.fft.ifft(np.fft.fft(y1)) - y1))

# Now, let's see compare what ifftshift to two things:
#   1. ifftshift(fftshift(np.fft.fft))
#   2. ifftshift(npfft)   - This one is the function I wrote where I append
#       the coefficient for mode +N as the complex conjugate of mode -N:
c1 = npfft(y1)
cp1 = np.fft.fftshift(np.fft.fft(y1))
if False:
    print("ifftshift on custom function:",np.fft.ifftshift(c1))
    print("ifft of custom function:", np.real(np.fft.ifft(np.fft.ifftshift(c1*(c1.size)))))
    print("ifft of numpy's fft:", np.real(np.fft.ifft(np.fft.ifftshift(cp1))))
    print("Original function y1:",y1)
    print("ifft of custom function, ignoring +N mode:", np.real(np.fft.ifft(np.fft.ifftshift(c1[:-1]*(c1.size-1)))))

    print("Are npfft and npifft consistent with each other?", isZero(npifft(npfft(y1))-y1) and isZero( npifft(npfft(y2))-y2 )   )

Are numpy's fft and ifft self-consistent?: True


In [143]:
# Checking if sp.fftconvolve gives the same results
N = 50 
x = (np.arange(-N,N)+0.5)/N*np.pi/alpha # x goes from -pi/alpha to pi/alpha (pi/alpha not included)
y1 = A*np.cos(M*alpha*x)
y2 = 2.*A*np.cos((M+1)*alpha*x)
c1 = npfft(y1)
c2 = npfft(y2)

print("Comparing performance of np.convolve, scipy.signal.convolve, scipy.signal.fftconvolve, and my custom functions at N=",N)
print("****************** \n np.convolve:")
%timeit np.convolve(c1,c2,mode='same')
print("****************** \n sig.convolve:")
%timeit sig.convolve(c1,c2,mode='same')
print("***************** \n sp.signal.fftconvolve:")
%timeit sig.fftconvolve(c1,c2, mode='same')
print("***************** \n npfft and npifft:")
%timeit npfft(npifft(c1)*npifft(c2))


N = 500 
x = (np.arange(-N,N)+0.5)/N*np.pi/alpha # x goes from -pi/alpha to pi/alpha (pi/alpha not included)
y1 = A*np.cos(M*alpha*x)
y2 = 2.*A*np.cos((M+1)*alpha*x)
c1 = npfft(y1)
c2 = npfft(y2)

print("\n\nComparing performance of np.convolve, scipy.signal.convolve, scipy.signal.fftconvolve, and my custom functions at N=",N)
print("Do np.convolve and sp.fftconvolve give the same result?:", 
      isZero(np.convolve(c1,c2,mode='same')- sig.fftconvolve(c1,c2,mode='same'))  )
print("Do np.convolve and custom convolve give the same result?:", 
      isZero( np.convolve(c1,c2,mode='same')- npfft(npifft(c1)*npifft(c2)) )  )
print("Do np.convolve and scipy.signal.convolve give the same result?:", 
      isZero( np.convolve(c1,c2,mode='same')- sig.convolve(c1,c2,mode='same') )  )


print("****************** \n np.convolve:")
%timeit np.convolve(c1,c2,mode='same')
print("****************** \n sig.convolve:")
%timeit sig.convolve(c1,c2,mode='same')
print("***************** \n sp.signal.fftconvolve:")
%timeit sig.fftconvolve(c1,c2, mode='same')
print("***************** \n npfft and npifft:")
%timeit npfft(npifft(c1)*npifft(c2))


N = 25000 
x = (np.arange(-N,N)+0.5)/N*np.pi/alpha # x goes from -pi/alpha to pi/alpha (pi/alpha not included)
y1 = A*np.cos(M*alpha*x)
y2 = 2.*A*np.cos((M+1)*alpha*x)
c1 = npfft(y1)
c2 = npfft(y2)

print("\n\nComparing performance of np.convolve, scipy.signal.convolve, scipy.signal.fftconvolve, and my custom functions at N=",N)
print("****************** \n np.convolve:")
%timeit np.convolve(c1,c2,mode='same')
print("****************** \n sig.convolve:")
%timeit sig.convolve(c1,c2,mode='same')
print("***************** \n sp.signal.fftconvolve:")
%timeit sig.fftconvolve(c1,c2, mode='same')
print("***************** \n npfft and npifft:")
%timeit npfft(npifft(c1)*npifft(c2))


Comparing performance of np.convolve, scipy.signal.convolve, scipy.signal.fftconvolve, and my custom functions at N= 50
****************** 
 np.convolve:
100000 loops, best of 3: 11.1 µs per loop
****************** 
 sig.convolve:
10000 loops, best of 3: 151 µs per loop
***************** 
 sp.signal.fftconvolve:
10000 loops, best of 3: 112 µs per loop
***************** 
 npfft and npifft:
10000 loops, best of 3: 153 µs per loop


Comparing performance of np.convolve, scipy.signal.convolve, scipy.signal.fftconvolve, and my custom functions at N= 500
Do np.convolve and sp.fftconvolve give the same result?: True
Do np.convolve and custom convolve give the same result?: True
Do np.convolve and scipy.signal.convolve give the same result?: True
****************** 
 np.convolve:
1000 loops, best of 3: 542 µs per loop
****************** 
 sig.convolve:
100 loops, best of 3: 12.7 ms per loop
***************** 
 sp.signal.fftconvolve:
1000 loops, best of 3: 239 µs per loop
***************** 
 np

## Figuring out how np.fft.fftn and np.fft.ifftn work

In [207]:
Nx = 3; Nz = 3
A = 2.; 
alpha = 3.; L = 2; beta= 1.5; M= 1
#y = vF.y; 
y = np.array([-1.,-0.3,0.3,1.])
y=y.reshape((1,1,y.size))
x = (np.arange(-Nx,Nx)+Nx)/Nx*np.pi/alpha # x goes from -pi/alpha to pi/alpha (pi/alpha not included)
z = (np.arange(-Nz,Nz)+Nz)/Nz*np.pi/beta
x = x.reshape((x.size,1,1))
z = z.reshape((1,z.size,1))

f1 = A*np.cos(L*alpha*x + M*beta*z)*(1.-y**2)
f2 = A*np.cos(alpha*x + 2*beta*z)*(1.-y**4)
print(f1.shape)

(6, 6, 4)


In [210]:
# The following code works
cp1 = np.fft.fftn(f1,axes=[0,1])/f1.size*4
c1 = np.fft.fftshift(cp1, axes=[0,1])
c1 = c1[1:,1:]

#np.real(np.fft.fftn(f1,axes=[0,1]))/f1.size
np.real(c1[:,:,1]/(1.-0.3*0.3))

array([[ -3.21951121e-17,   1.00000000e+00,   5.93067855e-17,
         -2.16893387e-16,  -1.69447959e-18],
       [ -1.52503163e-17,   3.70352346e-17,  -6.60847038e-17,
         -5.05910713e-17,   5.59178263e-17],
       [ -2.71116734e-17,   2.16893387e-16,   3.38895917e-18,
          2.71116734e-16,  -2.71116734e-17],
       [  5.59178263e-17,  -9.92356119e-18,  -6.60847038e-17,
         -3.63227549e-18,  -1.52503163e-17],
       [ -1.69447959e-18,  -6.50680161e-16,   5.93067855e-17,
          1.00000000e+00,  -3.21951121e-17]])

In [230]:
s1 = np.zeros((7,7,4), dtype=np.complex)
s1[1,2] = A/2.
s1[5,4] = A/2.
s1[:] *= 1.-y**2

s1 = s1[:-1,:-1]

%timeit p1 = np.fft.ifftn(  np.fft.ifftshift(s1,axes=[0,1]), axes=[0,1] )*(s1.shape[0]*s1.shape[1])

#c1[:,:,1], s1[1:,1:,1]
isZero(p1- f1)
norm((p1-f1).reshape(p1.size),2)

The slowest run took 6.22 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 79.1 µs per loop


1.4981301733398643e-14

## Convolution and FFT for 2-d arrays, Fourier-spectral on axis 0

This is the intermediate step before I finally move to my flowField objects

In [232]:
def convMan(ff1,ff2):
    """Returns convolution of flowField instances ff1 and ff2 along x and z as a 3d numpy array
    The assumption here is that they're both in spectral. 
    Convolution is computed by first doing an ifft of both arrays along axes given by argument axes,
        the arrays in physical space are multiplied, and the result is then fft'd
    I use numpy's fft, which is a bit unintuitive. I have to pad ff1 and ff2 before the ifft"""
    assert (ff1.nd==1) and (ff2.nd==1)
    # Padding with an extra wavenumber on both dimensions, this will be discarded later
    _f1 = ff1.slice(L=ff1.nx//2+1, M=ff1.nz//2+1)
    _f2 = ff2.slice(L=ff2.nx//2+1, M=ff2.nz//2+1)
    
    # Discarding the last positive modes, because numpy's fft doesn't like it if it was in there
    _f1 = _f1.view4d().copyArray()
    _f2 = _f2.view4d().copyArray()
    _f1 = _f1[0,:-1,:-1,0]
    _f2 = _f2[0,:-1,:-1,0]
    
    # Arranging modes in the order that numpy's fft likes, obtaining array in physical space
    ph1 = np.fft.ifftn(  np.fft.ifftshift(_f1, axes=[0,1]), axes=[0,1]  )*(_f1.shape[0]*_f1.shape[1])
    ph2 = np.fft.ifftn(  np.fft.ifftshift(_f2, axes=[0,1]), axes=[0,1]  )*(_f1.shape[0]*_f1.shape[1])
    
    # Convolution as product in physical space
    prod = ph1*ph2
    
    # Convolution by fft'ing product, and then shifting to the ordering I like
    conv = np.fft.fftshift(  np.fft.fftn(prod,axes=[0,1]), axes=[0,1] )/(_f1.shape[0]*_f1.shape[1])
    
    # Removing the last negative mode, which I only padded in.
    conv = conv[1:,1:]
    
    return conv      

In [233]:
#vF = mapData2ff(eps=0.01, g=0.8, Re=100., theta=0)[0]
vF = vf.copy()
#vF[:,:,:,1:] = 0.
u = vF.getScalar(); v=vF.getScalar(nd=1); w=vF.getScalar(nd=2)

%timeit vF.convNL().getScalar().view4d().copyArray()
convClassArr = vF.convNL().getScalar().view4d().copyArray()
convClassArr = convClassArr[0,:,:,0]

%timeit convMan(u,u.ddx())
convCustom = convMan(u,u.ddx()) + convMan(v,u.ddy()) + convMan(w,u.ddz())       
    
diff = convClassArr - convCustom
print(norm(convCustom.reshape(convCustom.size)), norm(diff.reshape(diff.size)))

1 loops, best of 3: 5.29 s per loop
10 loops, best of 3: 20.1 ms per loop
0.029560711972722295 8.267480741151165e-12


In [6]:
#(vf.convNL(fft=False) - vf.convNL()).norm()
start = time.time()
conv1= vf.convNL()
stop = time.time()
print(stop-start)
conv2 = vf.convNL(fft=False)
print(time.time()-stop)


0.33258962631225586
5.9157936573028564


In [8]:
Y = np.append(y1.reshape((y1.size,1)),y2.reshape((y2.size,1)),axis=1)
C = npfft2d(Y)

isZero(npconvolve2d(C,C) - npfft2d(Y*Y))

True

In [17]:
arr = np.array([0,1,2,3,-3,-2,-1]).reshape((7,1))
arr = arr*arr.reshape((1,7))
print(arr)
np.fft.fftshift(arr, axes=[0,1])

[[ 0  0  0  0  0  0  0]
 [ 0  1  2  3 -3 -2 -1]
 [ 0  2  4  6 -6 -4 -2]
 [ 0  3  6  9 -9 -6 -3]
 [ 0 -3 -6 -9  9  6  3]
 [ 0 -2 -4 -6  6  4  2]
 [ 0 -1 -2 -3  3  2  1]]


array([[ 9,  6,  3,  0, -3, -6, -9],
       [ 6,  4,  2,  0, -2, -4, -6],
       [ 3,  2,  1,  0, -1, -2, -3],
       [ 0,  0,  0,  0,  0,  0,  0],
       [-3, -2, -1,  0,  1,  2,  3],
       [-6, -4, -2,  0,  2,  4,  6],
       [-9, -6, -3,  0,  3,  6,  9]])

## Using scipy.signal.convolve2d (for each wall-normal node) to do the convolution


In [6]:
#vF = mapData2ff(eps=0.02,g=0.8,Re=100.,theta=0)[0]
vF = vf.copy()
vF[:,:,:,1:] = 0.
u = vF.getScalar()
uArr = u.view4d().copyArray(); duArr = u.ddx().view4d().copyArray()

uArr = uArr[0,:,:,0]; duArr = duArr[0,:,:,0]

%timeit vF.convNL().getScalar().view4d().copyArray()
convClassArr = vF.convNL().getScalar().view4d().copyArray()
print(convClassArr.shape)
convClassArr = convClassArr[0,:,:,0]

        
convConvolve = uArr.copy(); 
%timeit [sig.convolve2d(uArr[:,:,k],duArr[:,:,k],mode='same') for k in range(vF.N)] 

%timeit sig.convolve2d(uArr[:,:,5],duArr[:,:,5])

for k in range(vF.N): 
    convConvolve[:,:,k] = sig.convolve2d(uArr[:,:,k],duArr[:,:,k],mode='same') 
    
diff = convClassArr - convConvolve
print(norm(convConvolve.reshape(convConvolve.size)), norm(diff.reshape(diff.size)))



1 loops, best of 3: 236 ms per loop
(1, 47, 47, 1, 35)


NameError: name 'sig' is not defined

In [149]:
%timeit vf.slice(L=vf.nx//2+1,M=vf.nz//2+1)

100 loops, best of 3: 3.97 ms per loop
